# Milestone 1 without SparkSQL - Group 18

In [1]:
import pandas as pd

import findspark
findspark.init()
from pyspark.sql import functions as sf
from pyspark.sql.functions import when, col, trim, to_timestamp
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.functions import lead, lag
from pyspark.sql.window import Window
import pyspark.sql.functions as func
from pyspark.sql.functions import udf
import matplotlib.pyplot as plt
import matplotlib as mp
import numpy as np
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

# Loading the Data

In [4]:
raw_df = spark.read.option("header",True) \
    .csv("../MS1.csv",inferSchema=True).toDF(
  "Dummy", "Date", "Price", "Volume"
)
print('Number of rows in raw data:', raw_df.count())

raw_df.printSchema()

Number of rows in raw data: 38374197
root
 |-- Dummy: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Volume: string (nullable = true)



# View the Data

In [ ]:
raw_df.show()

# Data Pre-processing

In [ ]:
from pyspark.sql.functions import split
import pyspark.sql.functions as f
raw_df=raw_df.withColumn("Number", split(col("Dummy"), "[.]").getItem(0)).withColumn("Name", split(col("Dummy"), "[.]").getItem(1))

In [ ]:
raw_df=raw_df.drop(raw_df.Dummy)


In [ ]:
raw_df.show(100)
raw_df.count()

In [ ]:
raw_df

In [ ]:
#delete missing values
raw_df=raw_df.na.drop()
raw_df.count()

# Loading Time Series Data

In [ ]:
time_df = spark.read.option("header",True) \
    .csv("parking.csv")
print('Number of rows in raw data:', time_df.count())
time_df.show(10)
time_df.printSchema()

In [ ]:
#dropping the last three columns
col_drop=["BIN","BBL","NTA"]
for col in col_drop:
    time_df=time_df. drop(col)

#prinitng new schema
time_df.printSchema()
time_df.count()

In [ ]:
time_df = time_df.withColumnRenamed("Issue Date", "Date")
time_df.printSchema()

# Merging time series dataset with stocks dataset 

In [ ]:
df = raw_df.join(time_df,['Date'],how='inner').distinct()
df.show(10)

In [ ]:
df.printSchema()

In [ ]:
df.na.drop(how="any").show(truncate=False)

In [ ]:
df.show(10)

In [ ]:
df.count()

In [16]:
import numpy as np

def aggregate_avg(Y1, Y2):
    Y = []
    
    if len(Y1) != len (Y2):
        print("Arrays have different lengths!")
    else:
        for i in range(len(Y1)):
            Y.append(np.mean([Y1[i], Y2[i]]))
    
    return Y

def aggregate_min(Y1, Y2):
    Y = []
    
    if len(Y1) != len (Y2):
        print("Arrays have different lengths!")
    else:
        Y = np.minimum(Y1, Y2)
    
    return Y

def aggregate_max(Y1, Y2):
    Y = []
    
    if len(Y1) != len (Y2):
        print("Arrays have different lengths!")
    else:
        Y = np.maximum(Y1, Y2)
    
    return Y

def sum_of_multiplications(X, Y):
    sum_XY = 0
    if len(X) != len (Y):
        print("Arrays have different lengths!")
    else:
        for i in range(len(X)):
            mult = X[i] * Y[i]
            sum_XY = sum_XY + mult
    return sum_XY
        

In [18]:
import math
X = [3, 1, 4, 2]
Y1 = [1, 2, 1, 6]
Y2 = [3, 4, 1, 0]
Y = aggregate_max(Y1, Y2)

sum_X_YY = sum_of_multiplications(X, Y)
sum_X_squared = sum_of_multiplications(X, X)
sum_Y_squared = sum_of_multiplications(Y, Y)

cos_X_YY = sum_X_YY / (math.sqrt(sum_X_squared) * math.sqrt(sum_Y_squared))

cos_X_YY

0.6724213988522441

In [19]:
tau_avg = 0.3
tau_min = 0.3
tau_max = 0.3